---
# 落書き認識Webアプリを作ろう

- Author: Arata Furukawa ([github](https://github.com/ornew), [facebook](https://www.facebook.com/old.r.new))
- Contributor: Hideya Kawahara ([github](https://github.com/hideya))
---

このノートブックは、セミナーの資料として作成されています。ノートブックは、自由な編集、実行が可能です。Markdown形式でドキュメントも書き込めるため、必要に応じてメモを追記するなど、工夫してご利用ください。

編集も含めて保存したい場合は、画面上部のツールバーから、【File】タブを選び、【Download as】を選ぶことでローカルマシン上に保存することが可能です。

このノートブックは自由にご利用頂けますが、インターネット上への無断での転載だけはご遠慮くださいますようお願いします。

## ハンズオンの概要

Google が ["the Quick, Draw!"データセット](https://quickdraw.withgoogle.com/data) として公開している多量の落書きの画像データを用いてニューラルネットワークを訓練し、何の落書きかを認識するニューラルネットを生成します。そしてその学習済みニューラルネットワークを組み込んだ、落書きの分類をする Web App を作成します。以下は、最終的な Web App のスクリーンショットです。実際のアプリは [ここをクリックすることで実行](https://tfjs-doodle-recognition-pwa.netlify.com/) できます。

![](https://i.imgur.com/G6g18ap.png)

ニューラルネットワークのモデルとしては、手書き数字（[MNIST](https://en.wikipedia.org/wiki/MNIST_database)）の認識で実績のある [Convolutional Neural Network (CNN) ](https://en.wikipedia.org/wiki/Convolutional_neural_network) を用います。

モデルの構築と学習には、[AWS SageMaker](https://aws.amazon.com/jp/sagemaker/) 上で、
[TensorFlow](https://www.tensorflow.org/) を使用します。

Web App の構築には、JavaScript でニューラルネットワークの実装とブラウザ上での実行を可能とする [TensorFlow.js](https://js.tensorflow.org/) を利用します。

以下に続くノートブックで、それぞれの過程を詳しく説明します（なお、ニューラルネットワークの学習を行う際、数十円の課金が発生します）。

---
#### ご参考：ノートブックの操作方法

ノートブックは、自由な編集、実行が可能です。Markdown形式でドキュメントも書き込めるため、必要に応じてメモを追記するなど、工夫してご利用ください。

以下に良く使うキー操作を列挙します：

|キー操作| 説 明 | | キー操作 | 説 明 |
|--|--| |--|--|
| Enter | 編集モードに入る |　　　　| Esc → A | 新規セルを上に追加 | 
| Shift + Enter | セルを実行し / 編集モードから抜け、下のセルに移動 |　　　| Esc → B | 新規セルを下に追加 |
| Cntl + Enter | セルを実行する / 編集モードから抜ける |    |Esc → D, D | セルを削除 |
| Esc → M | セルをマークダウンモードに変更 |     | Esc → L | セルの行番号の表示・非表示 |
| Esc → Y | セルをコードモードに変更 |      | Esc → H | キーボード・ショートカットの一覧の表示 |

- ノートブックを初期状態に戻したい（全ての実行結果の消去とカーネルのリスタートをしたい）場合は、
    画面上部のツールバーから **Kernel → Restart & Clear Output** を選択します。

- 編集済みのノートブックをローカルにセーブしたい場合は、ツールバーから **File → Download as → Notebook**
    を選択します。

---

## モデルの概要

今回、認識する落書きは、以下の10クラス(種類)です。

1. りんご (apple)
2. ベッド (bed)
3. 猫 (cat)
4. 犬 (dog)
5. 目 (eye)
6. 魚 (fish)
7. 草 (grass)
8. 手 (hand)
9. アイスクリーム (ice cream)
10. ジャケット (jacket)

28x28ピクセルのグレースケール画像から、上記のいずれの落書きであるかを**確率的に**予測します。

![](./img/1.png)

### ディープラーニング

モデルは(ディープ)ニューラルネットワークで実装します。

ニューラルネットワークとは、生物のニューロン(神経細胞)のネットワークを数理モデルで模倣することで、特定の課題解決能力を機械的に学習する、機械学習アルゴリズムの一種です。深い層で構成されるニューラルネットワークの学習を行うことをディープラーニングといいます。

ディープラーニングにおけるモデルの学習は、以下の流れで行います。

- ⓪ モデルのパラメータを初期化する
- ① 学習用データに対する予測を計算する
- ② 教師ラベルと予測結果の誤差を計算する
- ③ 誤差を最小化するようにモデルのパラメータを更新する
- ④ **誤差が十分に小さくなるまで**①-③を繰り返す

![](./img/2.png)

## 実装の流れ

このノートブックでは、以下の手順で、ディープラーニングを用いた落書き(Doodle)認識を行うWebアプリを作成します。

1. ["the Quick, Draw!"データセット](https://quickdraw.withgoogle.com/data)を学習用データとして準備する
2. [TensorFlow](https://www.tensorflow.org/)で落書きを認識するディープニューラルネットワークのモデルを実装する
3. [Amazon SageMaker](https://aws.amazon.com/jp/sagemaker/)でモデルを学習する
4. [TensorFlow.js](https://js.tensorflow.org/)を使ったWebアプリに学習済みモデルを組み込む
5. [Amazon S3](https://aws.amazon.com/jp/s3/)でWebアプリを公開する

![](./img/3.png)

## 実装する

まず、作業に必要な Python のモジュールをノートブック・インスタンスに読み込みます。

実行ログの出力が始まるまで、少々（30秒程度）時間がかかりますので、しばらく反応がなくてもすこし様子を見てください。

In [ ]:
import tensorflow as tf

import six         # Python 2と3の互換性を保つためのライブラリです
import numpy as np # 行列などの科学数値計算をするためのライブラリです

import matplotlib.pyplot as plt # グラフを描画するライブラリです
%matplotlib inline

# 繰り返し処理の進捗をプログレスバーで表示するためのライブラリです
!pip install tqdm msgpack
from tqdm import tqdm_notebook as tqdm

### ① 学習用データを準備する

学習データは、Google社が[クリエイティブ・コモンズ ライセンス バージョン4.0](https://creativecommons.org/licenses/by/4.0/)で公開している["the Quick, Draw!"データセット](https://quickdraw.withgoogle.com/data)を利用します。

#### データをダウンロードする

データをダウンロードして、`./raw_data`ディレクトリに保存します。

ちなみに、Jupyterノートブックでは、「`!`」を先頭につけると、シェルコマンドを実行できます(Pythonの機能ではありません)。出力をPythonで使ったり、Pythonの変数を引数に使ったりも出来るので便利です。ここでは`wget`コマンドを使ってファイルをダウンロードします。

In [ ]:
URL = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap'
LABELS = [
    'apple', 'bed', 'cat', 'dog', 'eye',
    'fish', 'grass', 'hand', 'ice cream', 'jacket',
]

In [ ]:
!rm -rf ./data ./raw_data
!mkdir -p ./data ./raw_data
for l in LABELS:
    url = '{}/{}.npy'.format(URL, l)
    !wget -P raw_data "$url"

各ラベルのデータファイルがダウンロードできていることを確認します。

In [ ]:
!ls -l ./raw_data

ダウンロードしたデータを配列 (numpy.ndarray) に読み込みます。 

In [ ]:
raw_data = {label: np.load('raw_data/{}.npy'.format(label)) for label in tqdm(LABELS)}

各データの数を確認してみましょう。

In [ ]:
for label, data in six.iteritems(raw_data):
    print('{:10}: {}'.format(label, len(data)))

ためしに、1番目の「猫」の画像を表示してみます。

In [ ]:
plt.imshow(np.reshape(raw_data['cat'][0], [28, 28]), cmap='gray')
plt.show()

#### 学習用と評価用のデータを準備する

次に、データを学習用と評価用に分けます。

学習に使ったデータは、モデルがすでに「知っている」データなので、そのモデルが本当に役に立つのかを評価するためには学習に使っていない「未知のデータ」に対する精度を確認する必要があります。ですので、ダウンロードしたデータから、学習用と評価用の2種類のデータを予め準備します。

1. ダウンロードしたデータセットのうち1万件を取り出す *1
    - クラスごとに数にばらつきがあると、学習で用いられる頻度がクラスごとに変わってしまうため、揃えます
2. それぞれ画像データと教師ラベルの組み合わせに変換する
    - 教師ラベルは、クラスの名前(例:apple)ではなく、それぞれクラスごとにユニークな数字を割り当てます(下で確認)
3. 学習用と評価用に7:3で分ける
    - 学習に使われていないデータで精度の評価を行いたいため、3割を評価用のデータとして使います
4. ランダムにシャッフル

    *1 … もとは「10万」だったのですが、時間単価の安い（初期無料枠のある）「t2.medium」ノートブック・インスタンスで
    メモリ不足を起こさず扱えるように少なくしています。
    結果として、認識の精度（描画のブレへの対応力）は少々下がります。
    （SageMaker の料金一覧は[こちら](https://aws.amazon.com/jp/sagemaker/pricing/)）

学習に先立って、画像の各クラスを出力ニューロンと対応づけます。
ニューラルネットワーク (CNN) の認識結果は、各出力ニューロンの状態（０から１の値）により得られます。
そのため、それぞれの出力ニューロンを、判定するクラスと１対１に対応づけます。
つまり、10個の出力ニューロンを用意し、そのn番目のニューロンを、n番目のクラスと対応づけます。
ここでは、ラベル配列のインデクスをそのまま両者を対応付ける序数として使います。

In [ ]:
for i, label_name in enumerate(LABELS):
    print(u'出力ニューロン　番号: {}   クラス名: {}'.format(i, label_name))

次に、データを前処理して、ニューラルネットワークの訓練に用いるのに適した形式に変換します。

今回は、メモリ不足を起こさないように、また学習スピードを上げるために、最初にデータの個数を1万に削減します。
もともとは「10万」個のデータを使用していたのですが、
無料枠のある「t2.medium」ノートブック・インスタンスで処理をしてもメモリ不足を起こさないようにするために、
１万個に削減しています。 またこれにより、学習時間が短縮できます。
結果として、認識の精度（描画のブレへの対応力）は少々下がります。

In [ ]:
for label, data in six.iteritems(raw_data):
    raw_data[label] = raw_data[label][:10000]

for label, data in six.iteritems(raw_data):
    print('{:10}: {}'.format(label, len(data)))

各ピクセルの値が0から1に収まるように正規化し、学習用と評価用のデータに分割します。

In [ ]:
train_data = []
test_data = []
for label_name, value in six.iteritems(raw_data):
    label_index = LABELS.index(label_name)
    print('proccessing label class {}: "{}"'.format(label_index, label_name))
    # 各ピクセルの値を、0-255から0-1に修正します
    value = np.asarray(value) / 255.
    # 7万件を学習用のデータとして画像データと教師ラベルの組み合わせにしてリストに追加します
    train_data.extend(zip(value[:7000], np.full(7000, label_index)))
    # 3万件を評価用のデータとして画像データと教師ラベルの組み合わせにしてリストに追加します
    test_data.extend(zip(value[7000:10000], np.full(3000, label_index)))
np.random.shuffle(train_data)
np.random.shuffle(test_data)

次に、これら学習用と評価用のデータを、ニューラルネットワークの訓練での利用に適した
TFRecord 形式のファイルに変換して出力します。
TFRecord は [Protocol Buffers](https://developers.google.com/protocol-buffers/) というフォーマットを用いたデータファイルで、構造化されている・圧縮効率が高い・読み書きの速度が非常に速い・非同期のストリーミング読み込みが可能
とった長所があり、機械学習で用いられる大規模データセットの保存に向いています。

まず、ヘルパー関数を定義します。

In [ ]:
train_filename = './data/train.tfr'
test_filename  = './data/test.tfr'

def get_example_proto(image, label):
    """
    画像とラベルをProtocol Buffers形式のtf.train.Exampleに変換します
    """
    return tf.train.Example(features=tf.train.Features(feature={
        'image' : tf.train.Feature(float_list=tf.train.FloatList(value=image)),
        'label' : tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
    })).SerializeToString()

以下の変換処理は30秒ほどかかります。

In [ ]:
%%time
tfr_options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
with tf.python_io.TFRecordWriter(train_filename, tfr_options) as train_tfr, \
     tf.python_io.TFRecordWriter(test_filename, tfr_options) as test_tfr:
    print('Converting train data...')
    for data, label in tqdm(train_data):
        train_tfr.write(get_example_proto(data, [label]))
    print('Converting test data...')
    for data, label in tqdm(test_data):
        test_tfr.write(get_example_proto(data, [label]))

`train.tfr`と`test.tfr`が生成されていれば成功です。

In [ ]:
!ls -l data

これで、データの準備が完了しました。

生成したデータは、学習モデルの設定を終えた後、学習プロセスを開始する前に、
S3 にアップロードして、学習用インスタンスからアクセスできるようにします。

### ② TensorFlowでモデルの定義プログラムを実装する

モデルの実装には、[TensorFlow](https://www.tensorflow.org/)を利用します。TensorFlowは、Google社が主体となって開発している、オープンソースの汎用的な分散数値演算ライブラリです。TensorFlowにはディープラーニング向けのライブラリが用意されています。GitHubのスターは10万近くあり、現在世界で最も人気のディープラーニングフレームワークとも言われています。

以下の4つの関数を定義したプログラムを用意すると、Amazon SageMakerを使ってモデルの学習を行うことができます。

```python
def train_input_fn(training_dir, hyperparameters):
    """
    学習用の入力データを読み込みます。
    
        training_dir: 学習の実行時に指定したS3のファイルがこの文字列のディレクトリにマウントされています。
        hyperparameters: 学習の実行時に指定したハイパーパラメータが渡されます。
        
    基本的には、以下のことを実装するだけです。
    ① hyperparametersで指定した挙動に従って、
    ② training_dirから学習データを読み込み、データを返す。
    """

def eval_input_fn(training_dir, hyperparameters):
    """
    評価用の入力データを読み込みます。
    やることはtrain_input_fnと同じですが、評価用のデータを読み込むことや、
    評価用に挙動を変える(例えば評価データはシャッフルしないなど)ことが可能です。
    """

def serving_input_fn(hyperparameters):
    """
    モデルの入力データの形式を定義します。
    サービングと付いている通り、SageMakerでAPIサーバにデプロイしたときの入力データ定義にもなります。
    """

def model_fn(features, labels, mode, hyperparameters):
    """
    モデルの定義をします
    
        features: モデルの入力と成る特徴データです *_input_fnで返した値がそのまま渡されます。
        labels: モデルの教師ラベルデータです。
        mode: モデルの実行モードです。実行モードには「学習」「評価」「推論」があり、挙動を切り替えることが可能です。
        hyperparameters: 実行時に指定したハイパーパラメータが渡されます。
    """
```

最後の`model_fn`が、その名のとおり、ニューラルネットワークの定義の本体です。

`model_fn`の中では、以下の3つを定義します。

1. **モデル**: ニューラルネットワーク
2. **誤差**: 教師データと予測結果がどの程度違ったのかを定式化する
3. **最適化アルゴリズム**: 誤差を最小化するようにモデルを最適化するアルゴリズム

つまり、データの入力方法と、上記3つのモデル関連の定義を行うだけで、機械学習を行うことができてしまいます。

今回、セミナー用のモデル定義は予め実装してあります(`src/doodle.py`ファイル)。

以下を実行して、その内容を確認してみましょう。
コメントなどを含めても200行弱程度しかありません。
コードには多くのコメントが付けてありますので、ざっと目を通してみてください。

In [ ]:
!cat src/doodle.py

ニューラルネットワークのモデルの定義や学習に関する詳細は、別途ノートブック`model.ipynb`で解説しています。ニューラルネットワークの実装に興味がある方はそちらをご参照ください。

### ③ Amazon SageMakerでモデルを学習する

Amazon SageMaker SDKを使い、ここまでで準備したデータとプログラムを指定して学習を実行します。

![](img/4.png)

#### 設定情報を定義する

モデルの学習を始めるにあたり、学習に使用するデータの保存先などの設定情報を変数で定義します。
ここで、学習ジョブの名前を定義しますので、もし再度、学習を繰り返したい場合には、ここから以下を再実行します。

In [ ]:
import sagemaker
from datetime import datetime
import six

role = sagemaker.get_execution_role()
session = sagemaker.Session()
bucket = session.default_bucket()
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

def _s3(path):
    return 's3://{}/doodle-{}/model/{}'.format(bucket, timestamp, path)

data_key_prefix =  'doodle-{}/model/data'.format(timestamp)

config = dict(
    data_dir        = _s3('data'),
    output_path     = _s3('export'),
    checkpoint_path = _s3('ckpt'),
    code_location   = _s3('src'),
    public_dir      = _s3('public'),
    job_name        = 'doodle-training-job-{}'.format(timestamp)
)

確認のため、設定した変数を表示します。

In [ ]:
for k, v in six.iteritems(config):
    print('key: {:20}, value: {:20}'.format(k, v))

#### S3 にデータをアップロードする

上記で設定した S3 パスに、①で作成したデータセットをアップロードし、学習インスタンスが学習データにアクセスできるようにします。

In [ ]:
uploaded_data_dir = session.upload_data(
    'data',                     # ローカルディレクトリ
    bucket=bucket,    # アップロードするS3バケット名
    key_prefix=data_key_prefix) # アップロードするパスのプリフィクス

# 設定と同じ場所になったか念のため確認します
assert uploaded_data_dir == config['data_dir']

これで、学習用と評価用のデータの準備がおわりました。
次に、学習を行うモデルを構築します。

モデルの学習では、「エスティメータ(Estimator)」を利用します。 エスティメータとは、モデルの学習や評価、保存やデプロイといった一連の処理を簡便に行うための、高レベルのインターフェイスです。

用意したパスなどを設定として渡して、エスティメータを作成します。

エスティメータへのパラメータとして、`entry_point`に`doodle.py`が指定されていることに注目してください。
この Python のプログラム`doodle.py`で、②で述べた、ニューラルネットワークのモデルやそれに関わるシステムの挙動が定義されています。

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # ハイパーパラメータ
    # ②で定義したプログラムの各関数の引数に渡されます
    # プログラムの挙動を切り替えるのに利用できます
    hyperparameters={
        'save_summary_steps': 100,
        'throttle_secs': 120,
    },
    
    # 先程設定した、各データの保存先のパス
    output_path     = config['output_path'],
    checkpoint_path = config['checkpoint_path'],
    code_location   = config['code_location'],
    
    # 学習用プログラムに関する設定
    source_dir='./src',      # 学習用のプログラムが保存されたローカルディレクトリ
    entry_point='doodle.py', # ②で定義した学習用プログラムのファイル名
    framework_version='1.6', # 利用したいTensorFlowのバージョン
    
    # 学習と評価の回数
    training_steps=10000,
    evaluation_steps=1000,
    
    # AWSでの実行に関する設定
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge') # ml.p2.xlargeはGPUの搭載されたインスタンスです

このエスティメータに対して、学習用データのパス名を指定して`fit`関数を呼び出すと、学習ジョブが作成され、クラウド上でモデルの学習を実行します。

この学習には10分弱かかります。
その間、学習中の状態を確認するために、ノートブック上で
[TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard)
を起動します
（そのために`run_tensorboard_locally`引数に`True`を渡します）。

なお、この学習では、１回で、数十円の課金が発生します
（ご参考：「[Amazon SageMaker の料金](https://aws.amazon.com/jp/sagemaker/pricing/)」）。

それでは、以下のセルで、学習をスタートしてみましょう。

In [ ]:
%%time
estimator.fit(config['data_dir'], job_name=config['job_name'],
              wait=True, run_tensorboard_locally=True)

ノートブック上で実行される TensorBoard は、学習経過を表示するように設定してあります。

[ここをクリックすることにより](/proxy/6006/)、TensorBoard の画面がブラウザで開き、学習経過が確認できます
（実行ログ中に表示される`http://localhost:6006`ではアクセスできません。
`https://(ノートブックのURL)/`[proxy/6006/](/proxy/6006/) にアクセスする必要があります）。

学習のセットアップをしている最初のうちは「No dashboards are active for the current data set」とだけ表示されますが、
４〜５分経って、上のセルにログが出力され始めると、学習経過を示すグラフが表示されるようになります。
ログが表示されはじめるのを待ってから、TensorBoard のページを確認してみてください。

上の学習ジョブが終了したら、学習したモデルをチェックしてみましょう。

学習済みモデルのファイルは、エスティメータのoutput_path引数で指定した場所に保存されています。

In [ ]:
output_dir_url = '{}/{}/output/'.format(config['output_path'], config['job_name'])
!echo $output_dir_url
!aws s3 ls $output_dir_url

### ④ 学習したモデルをダウンロードして、Webアプリに組み込む

学習したモデルは、エスティメータの`output_path`引数で指定した場所にGZIP圧縮されたTarアーカイブとして保存されています。中身はTesnorFlow SavedModelと呼ばれるデータ形式です。

![](img/5.png)

このモデルデータを使えば、Pythonで実行したり、TensorFlow ServingでAPIサーバを構築したり、TensorFlow Liteを使ってAndroidやiOSで実行したりすることが可能です。

今回は、TensorFlow.jsを使って、Webブラウザ上でモデルの推論を実行してみましょう。

まず、学習済みモデルデータをS3からノートブック・インスタンスにダウンロードして解凍します。

In [ ]:
model_url = '{}/{}/output/model.tar.gz'.format(config['output_path'], config['job_name'])

In [ ]:
!rm -rf ./export ./model.tar.gz
!aws s3 cp "$model_url" ./model.tar.gz
!tar xvzf ./model.tar.gz

次に、TensorFlow.js で読み込め、推論が実行できる形式に、学習済みモデルデータのフォーマットを変更します。

そために、変換用ツールをインストールし、これを上で解凍したモデルデータに対して適用します。

In [ ]:
# 変換用ツールをインストールします
!pip install tensorflowjs

In [ ]:
# 変換したモデルの保存先ディレクトリを作成します
!rm -rf ./webapp
!mkdir -p ./webapp/model

# 変換ツールを実行します
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='probabilities,classes' \
    --saved_model_tags=serve \
    ./export/Servo/* \
    ./webapp/model

これで、`./webapp/model`ディレクトリにTensorFlow.jsで読み込めるモデルデータが生成されました！

In [ ]:
!ls -l ./webapp/model/

このモデルデータを TensorFlow.js で読み込み、推論を実行します。

以下に、読み込みおよび推論実行コードの主要部をあげます：

```javascript
// ライブラリを読み込みます
import * as tf from '@tensorflow/tfjs-core';
import {loadFrozenModel} from '@tensorflow/tfjs-converter';
...
// モデルを読み込みます
const model = await loadFrozenModel(modelUrl, weightsUrl);
...
// モデルで推論を実行し、結果を獲得します
const output = model.execute({'image_1': input/* 画像データ */}, 'probabilities');
const probabilities = output.dataSync();
```

非常に簡単です！
とはいえ実際には、諸設定を行ったりアプリケーションとしての体裁を整えたりする「ガワ」を作らねばなりません。
その詳細な説明は、ここでは割愛します。
その代わり、既に用意されている、モデルデータを簡単に組み込めるWebアプリケーションのテスト用コードに、
学習済みのモデルデータを組み込んで実行することにより、データの検証をしてみましょう。

まず、作成済みのWebアプリケーションのZIPをダウンロードして展開します。

In [ ]:
!wget -O webapp.zip https://github.com/hideya/tfjs-doodle-recognition-pwa/releases/download/0.0.3/nonpwa-webapp.zip
!unzip webapp.zip -d webapp

これで、`webapp`ディレクトリ以下にWebアプリケーションに必要なものが全て揃いました！

確認してみましょう。

In [ ]:
!ls -Rl ./webapp/

なお、このWebアプリケーションのソースコードは
[https://github.com/hideya/tfjs-doodle-recognition-pwa/tree/simple](https://github.com/hideya/tfjs-doodle-recognition-pwa/tree/simple)
にて公開しています（simpleブランチです。masterブランチはsimpleバージョンを [PWA](https://developers.google.com/web/progressive-web-apps/) 化したものになります）。

また、[Vue.js](https://vuejs.org/) を用いたプログラムサンプルも [maru-labo/doodle/examples/tensorflow_js](https://github.com/maru-labo/doodle/tree/master/examples/tensorflow_js) にて公開しています。ぜひ参考にしてください。

### ⑤ WebアプリをS3でホスティングして公開する

`webapp`ディレクトリに必要なものが揃ったので、Web上に公開してみましょう。S3の静的ホスティング機能を使うと簡単にWebアプリケーションを公開できます。`aws s3 sync`コマンドで`webapp`ディレクトリを`public_dir`変数に格納したURLにアップロードします。

In [ ]:
public_dir = config['public_dir']
!aws s3 sync ./webapp $public_dir

これで必要なファイルが S3 にアップロードされました。

このアプリを公開し、ブラウザからアクセスできるようにするためには、ファイルが格納された S3 のバケットの「Access」を「Public」にする必要があります（必要なファイルのみをPublicにする方法もあるのですが、今回は簡便のためバケット全体をPublicに設定します）。

そのためには、まず、下記セルを実行して表示される URL をクリックして、学習結果が格納されたバケットを開きます

In [ ]:
print('https://s3.console.aws.amazon.com/s3/buckets/{}'.format(bucket))

まず「プロパティ Properties」タブを開き、「Static website hosting」を選択し、さらに「このバケットを使用してウェブサイトをホストする Use this bucket to host a website」を選択します。
「インデックスドキュメント Index document」には「index.html」と書き込みます。その他は空欄のままでOKです。
「保存 Save」をクリックします。

さらに、もう一つ、設定が必要です。

「アクセス権限 Permissions」タブを開きます。
次に、タブのすぐ下に表示されるメニューから「バケットポリシー Buket Policy」を選択します。

「バケットポリシーエディター Bucket policy editor」が開いたら、以下のテキストをコピペし、くわえて、「sagemaker-us-west-2-000000000000」の部分を、
今対象としているバケット名に合うように書き換えます。
「保存 Save」をクリックします。

「このバケットにはパブリックアクセス権限があります This bucket has public access」と警告が表示され、「バケットポリシー Buket Policy」の下に「パブリック Public」と表示されれば完了です。

```
{
  "Version": "2012-10-17",
  "Statement": [
     {
    "Sid": "PublicReadGetObject",
    "Effect": "Allow",
    "Principal": "*",
    "Action": "s3:GetObject",
    "Resource": "arn:aws:s3:::sagemaker-us-west-2-000000000000/*"
     }
  ]
}
```

以上で、S3 バケットの設定は完了です。これで、アップロードされたWebアプリケーションがブラウザからアクセスできるようになりました。

それでは早速、アップロードされたWebアプリケーションの動作を確認してみましょう！

下記セルを実行するとURLが表示されますので、クリックして開いてみてください。
大きなモデルデータを読み込む必要があるため、起動に少々時間がかかります。

In [ ]:
print('https://s3-{}.amazonaws.com/{}/index.html'.format(session.boto_region_name, public_dir[5:]))

print('\nもし上のURLで「Error PermanentRedirect」が発生してうまく表示されない場合は、下のURLを試してみてください。')
print('https://s3.amazonaws.com/{}/index.html'.format(public_dir[5:]))

本ノートブック冒頭のスクリーンショットのようなアプリが、ちゃんと表示されましたでしょうか？
落書きの認識精度はいかがでしょうか？

これで、本ハンズオンは終了です。おつかれさまでした。

認識率の改善を目指して学習データの数を増やしたり、学習パラメータを変更したり、いろいろ実験してみてください。

### 後始末

実験の後、インスタンスを動いたままにしていたり、巨大なデータを放置していると、思わぬ課金が発生することがあります。

実験がひととおり終わったら、以下の後始末をすることをおすすめします：

- ノートブックを閉じたら、ノートブック・インスタンスを停止する（これは忘れやすいので気をつけてください）。
- 実験等で多量に溜まったデータファイルのうち不要なものは、S3 やノートブックインスタンスから削除する。
- Public になっている S3 の設定を Private にもどす（外部からの意図しないアクセスによるデータ送信の課金を避ける）。

課金の状態は、「Billing & Cost Management Dashboard」で簡単に確認することができます。

こちらのURL https://console.aws.amazon.com/billing/home をクリックして、確認してみてください
（前日までの利用料が表示されますので、今日の分を知りたい場合は、翌日まで待ってからチェックしてみてください）。 

## まとめ

- ✔ 簡単なデータセットを作りました
- ✔ SageMakerでモデルを学習しました
- ✔ SageMakerで学習したモデルをWebアプリケーションで実行しました

なお、落書き認識モデルについては、本日使用したサンプルを含めて全て[GitHub](https://github.com/maru-labo/doodle)上で公開していますので、より詳しい情報をご希望の方はぜひご参照ください。今後、LiteのサンプルやServingの使い方などもリポジトリに追加する予定です。MITライセンスですので、ご自由にご利用いただけます。お気軽にIssueやPull Requestをお寄せくださいませ。